In [210]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [211]:
dftrain = pd.read_csv("train.csv")
dftest = pd.read_csv("test.csv")
dftrain.head(5)

,id,bone_length,rotting_flesh,hair_length,has_soul,color,type
0,0,0.354512,0.350839,0.465761,0.781142,clear,Ghoul
1,1,0.575560,0.425868,0.531401,0.439899,green,Goblin
2,2,0.467875,0.354330,0.811616,0.791225,black,Ghoul
3,4,0.776652,0.508723,0.636766,0.884464,black,Ghoul
4,5,0.566117,0.875862,0.418594,0.636438,green,Ghost


In [212]:
test_ids = dftest.id
X_test = dftest.drop(columns=['id'])
X_test.head(5)

,bone_length,rotting_flesh,hair_length,has_soul,color
0,0.471774,0.387937,0.706087,0.698537,black
1,0.427332,0.645024,0.565558,0.451462,white
2,0.549602,0.491931,0.660387,0.449809,black
3,0.638095,0.682867,0.471409,0.356924,white
4,0.361762,0.583997,0.377256,0.276364,black


In [213]:
Y_train = dftrain.type
X_train = dftrain.drop(['type','id'],1)
X_train.head()

,bone_length,rotting_flesh,hair_length,has_soul,color
0,0.354512,0.350839,0.465761,0.781142,clear
1,0.575560,0.425868,0.531401,0.439899,green
2,0.467875,0.354330,0.811616,0.791225,black
3,0.776652,0.508723,0.636766,0.884464,black
4,0.566117,0.875862,0.418594,0.636438,green


In [214]:
#from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV

In [215]:
print(X_train.shape,X_test.shape)
X_all = X_train.append(X_test)
X_all.shape
X_all=pd.get_dummies(X_all,columns=['color'],prefix=['color'],prefix_sep='_')
X_train = X_all[:371]
X_test = X_all[371:]
print(X_train.shape,Y_train.shape,X_test.shape,test_ids.shape)
X_train.head()

(371, 5) (529, 5)
(371, 10) (371,) (529, 10) (529,)


,bone_length,rotting_flesh,hair_length,has_soul,color_black,color_blood,color_blue,color_clear,color_green,color_white
0,0.354512,0.350839,0.465761,0.781142,0,0,0,1,0,0
1,0.575560,0.425868,0.531401,0.439899,0,0,0,0,1,0
2,0.467875,0.354330,0.811616,0.791225,1,0,0,0,0,0
3,0.776652,0.508723,0.636766,0.884464,1,0,0,0,0,0
4,0.566117,0.875862,0.418594,0.636438,0,0,0,0,1,0


In [218]:
"""
from sklearn.svm import SVC
params = {
    'kernel':['linear','rbf'],
    'C':[0.5,1.0,1.5]
} 
svc = SVC()
gridsearch_svm = GridSearchCV(svc,param_grid=params,scoring='accuracy',cv=5,verbose=3,n_jobs=5)
gridsearch_svm.fit(X_train,Y_train)
best_svm = gridsearch_svm.best_estimator_
print(best_svm)
best_svm.fit(X_train,Y_train)
best_svm.score(X_train,Y_train)
"""
''

''

In [217]:
"""
Y_train = pd.DataFrame({
    'type':Y_train
})
Y_train.head()
Y_train[Y_train['type']=='Ghoul'] = 1
Y_train[Y_train['type']=='Goblin'] = 2
Y_train[Y_train['type']=='Ghost'] = 3
Y_train.head()
"""
''

''

In [219]:
"""
from catboost import CatBoostClassifier,Pool
params= {
    'iterations':500,
    'learning_rate':0.1,
    'depth':None,
    'random_seed':5,
    'logging_level':'Verbose',
    'use_best_model':True,
    'loss_function':'MultiClass'
}
clf = CatBoostClassifier(**params)
from sklearn.model_selection import train_test_split
X_train,X_validation,Y_train,Y_validation = train_test_split(X_train,Y_train,test_size=0.3)
train_pool = Pool(X_train,Y_train,cat_features=[4],has_header=True)
validation_pool = Pool(X_validation,Y_validation,cat_features=[4],has_header=True)
clf.fit(train_pool,eval_set=validation_pool,verbose=True)
clf.score(X_validation,Y_validation)
"""
''

''

In [223]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
param_grid={
    'n_estimators':[10,30,50,100],
    'max_depth':[3,6,10,None]
}
gridsearch_rf = GridSearchCV(clf,param_grid=param_grid,scoring='accuracy',n_jobs=4,cv=4,verbose=3).fit(X_train,Y_train)
best_rf = gridsearch_rf.best_estimator_

Fitting 4 folds for each of 16 candidates, totalling 64 fits
[CV] max_depth=3, n_estimators=10 ....................................
[CV] max_depth=3, n_estimators=10 ....................................
[CV] max_depth=3, n_estimators=10 ....................................
[CV] max_depth=3, n_estimators=10 ....................................
[CV]  max_depth=3, n_estimators=10, score=0.717391304347826, total=   0.1s
[CV]  max_depth=3, n_estimators=10, score=0.6105263157894737, total=   0.1s
[CV] max_depth=3, n_estimators=30 ....................................
[CV] max_depth=3, n_estimators=30 ....................................
[CV]  max_depth=3, n_estimators=10, score=0.7065217391304348, total=   0.1s
[CV] max_depth=3, n_estimators=30 ....................................
[CV]  max_depth=3, n_estimators=10, score=0.6521739130434783, total=   0.1s
[CV] max_depth=3, n_estimators=30 ....................................
[CV]  max_depth=3, n_estimators=30, score=0.6421052631578947, total=

[Parallel(n_jobs=4)]: Done  40 tasks      | elapsed:    3.4s


[CV]  max_depth=10, n_estimators=50, score=0.6630434782608695, total=   0.4s
[CV] max_depth=None, n_estimators=10 .................................
[CV]  max_depth=10, n_estimators=100, score=0.7052631578947368, total=   0.5s
[CV] max_depth=10, n_estimators=100 ..................................
[CV]  max_depth=None, n_estimators=10, score=0.6421052631578947, total=   0.1s
[CV] max_depth=None, n_estimators=10 .................................
[CV]  max_depth=10, n_estimators=50, score=0.6630434782608695, total=   0.4s
[CV] max_depth=None, n_estimators=10 .................................
[CV]  max_depth=None, n_estimators=10, score=0.6847826086956522, total=   0.1s
[CV] max_depth=None, n_estimators=30 .................................
[CV]  max_depth=10, n_estimators=100, score=0.7065217391304348, total=   0.7s
[CV]  max_depth=None, n_estimators=10, score=0.7065217391304348, total=   0.1s
[CV] max_depth=None, n_estimators=10 .................................
[CV] max_depth=10, n_estima

[Parallel(n_jobs=4)]: Done  64 out of  64 | elapsed:    5.8s finished


In [224]:
#predictions = best_svm.predict(X_test)
predictions = best_rf.predict(X_test)

"""
pred = pd.DataFrame(predictions)
pred[pred[0]==1] = 'Ghoul'
pred[pred[0]==2] = 'Goblin'
pred[pred[0]==3] = 'Ghost'
pred.head()
"""

"\npred = pd.DataFrame(predictions)\npred[pred[0]==1] = 'Ghoul'\npred[pred[0]==2] = 'Goblin'\npred[pred[0]==3] = 'Ghost'\npred.head()\n"

In [228]:
res = pd.DataFrame({
    "id":test_ids,
    "type":predictions
})
print(res.shape)
res.head(5)

(529, 2)


,id,type
0,3,Ghoul
1,6,Goblin
2,9,Ghoul
3,10,Ghost
4,13,Ghost


In [229]:
res.to_csv("sol.csv",index=False,header=['id','type'])